# Forward Volatility

* fut_near_month_close, fut_far_month_close : 근월물, 원월물의 KOSPI200 선물 종가

* strike_price : 콜옵션 행사가

* near_month_expiry, far_month_expiry : 근월물, 원월물 만기일

* LTP_near_month, LTP_far_month : 각 근월물, 원월물의 행사가(strike) 9800 call 의 마지막 거래가격 (LTP)

In [9]:
import mibian as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

In [10]:
nifty_data = pd.read_csv('kospi200.csv', index_col=1, parse_dates=['date', 'far_month_expiry', 'near_month_expiry'])

In [11]:
nifty_data

,symbol,fut_near_month_close,fut_far_month_close,strike_price,near_month_expiry,LTP_near_month,far_month_expiry,LTP_far_month
date,,,,,,,,
2018-08-20,KOSPI200,288.95,288.95,290,2018-09-13,3.29,2018-10-11,4.36


## 잔존일수 계산

In [12]:
nifty_data['near_month_days_to_expiry'] = (nifty_data['near_month_expiry'] - nifty_data.index).dt.days
nifty_data['far_month_days_to_expiry'] = (nifty_data['far_month_expiry'] - nifty_data.index).dt.days
nifty_data.loc[:, ['near_month_expiry', 'near_month_days_to_expiry', 'far_month_expiry', 'far_month_days_to_expiry']].head()

,near_month_expiry,near_month_days_to_expiry,far_month_expiry,far_month_days_to_expiry
date,,,,
2018-08-20,2018-09-13,24,2018-10-11,52


In [13]:
nifty_data.head(2)

,symbol,fut_near_month_close,fut_far_month_close,strike_price,near_month_expiry,LTP_near_month,far_month_expiry,LTP_far_month,near_month_days_to_expiry,far_month_days_to_expiry
date,,,,,,,,,,
2018-08-20,KOSPI200,288.95,288.95,290,2018-09-13,3.29,2018-10-11,4.36,24,52


# 내재변동성 (IV) 계산

BS model 을 이용하여 내재 변동성 계산. 입력 data 는 선물 종가, 행사가, 이자율, 잔존일수, 콜옵션 가격 이다.

선물가격을 기초자산 가격으로 하므로, 이자율은 0 으로 한다.

In [14]:
nifty_data['IV_near_month'] = 0
nifty_data['IV_far_month'] = 0

for row in range(len(nifty_data)):
    nifty_data.iloc[row, nifty_data.columns.get_loc('IV_near_month')] = m.BS([
        nifty_data.iloc[row]['fut_near_month_close'], nifty_data.iloc[row]['strike_price'], 0,
        nifty_data.iloc[row]['near_month_days_to_expiry']
    ],
    callPrice=nifty_data.iloc[row]['LTP_near_month']
    ).impliedVolatility
    
    nifty_data.iloc[row, nifty_data.columns.get_loc('IV_far_month')] = m.BS([
        nifty_data.iloc[row]['fut_far_month_close'], nifty_data.iloc[row]['strike_price'], 0,
        nifty_data.iloc[row]['far_month_days_to_expiry']
    ],
    callPrice=nifty_data.iloc[row]['LTP_far_month']
    ).impliedVolatility

nifty_data.loc[:, ['IV_near_month', 'IV_far_month']].head(2)

,IV_near_month,IV_far_month
date,,
2018-08-20,12.817383,11.169434


## forward volatility 계산

In [15]:
# 근월물과 원월물에 대한 각각의 variance 계산
nifty_data['variance_near_month'] = (nifty_data.IV_near_month**2/365) * nifty_data.near_month_days_to_expiry
nifty_data['variance_far_month'] = (nifty_data.IV_far_month**2/365) * nifty_data.far_month_days_to_expiry

# 두 variance 의 차이를 계산
nifty_data['variance_diff'] = nifty_data.variance_far_month - nifty_data.variance_near_month
nifty_data['forward_variance_days'] = nifty_data.far_month_days_to_expiry - nifty_data.near_month_days_to_expiry

# forward variance 를 forward volatility 로 convert
nifty_data['forward_volatility'] = (nifty_data.variance_diff * 365 / nifty_data.forward_variance_days)**0.5

In [16]:
nifty_data.head(2)

,symbol,fut_near_month_close,fut_far_month_close,strike_price,near_month_expiry,LTP_near_month,far_month_expiry,LTP_far_month,near_month_days_to_expiry,far_month_days_to_expiry,IV_near_month,IV_far_month,variance_near_month,variance_far_month,variance_diff,forward_variance_days,forward_volatility
date,,,,,,,,,,,,,,,,,
2018-08-20,KOSPI200,288.95,288.95,290,2018-09-13,3.29,2018-10-11,4.36,24,52,12.817383,11.169434,10.802321,17.773493,6.971171,28,9.532796


# 거래 signal 생성

forward volatility 가 근월물의 volatility 보다 크면 원월물이 근월물 옵션보다 비싸다는 것을 나타내므로 원월물을 팔고, 근월물을 산다.

-1 을 signal column 에 저장함.

In [17]:
nifty_data['signal'] = np.where(nifty_data.forward_volatility > nifty_data.IV_near_month, -1, 1)
nifty_data.loc[:, ['forward_volatility', 'IV_near_month', 'signal']].head()

,forward_volatility,IV_near_month,signal
date,,,
2018-08-20,9.532796,12.817383,1


# Strategy returns

Strategy return 은 원월물과 근월물의 수익 차이를 계산하여 전일자의 trading signal (+1, -1) 을 곱하여 구한다.

In [18]:
nifty_data['far_month_ret'] = np.log(nifty_data.LTP_far_month/nifty_data.LTP_far_month.shift(1))
nifty_data['near_month_ret'] = np.log(nifty_data.LTP_near_month/nifty_data.LTP_near_month.shift(1))

nifty_data['strategy_return'] = (nifty_data.far_month_ret - nifty_data.near_month_ret) * nifty_data.signal.shift(1)

In [19]:
plt.figure(figsize=(10, 6))
plt.xlabel('Date')
plt.ylabel('Cumulative Strategy Returns')
plt.plot(nifty_data.strategy_return.cumsum())
plt.grid()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001BF17839D08> (for post_execute):


ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 720x432 with 1 Axes>

이 전략은 1 개월간 약 25% 의 return 을 보임